## 데이터 불러오기

In [375]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [376]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [377]:
# pip install sentence_transformers

In [378]:
# pip install textblob

In [379]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

import nltk
from nltk.tokenize import RegexpTokenizer       
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4") 

client=pymongo.MongoClient("192.168.10.240",27017)
db= client['healthcare_sideproject']
FHP_article = db['naver_FHP_data']
FHP_article = FHP_article.find()
FHP_article = pd.DataFrame(FHP_article)
FHP_article = FHP_article.loc[:,'title':]
FHP_article

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,title,artical_date,abstract_list
0,Two Methods of Forward Head Posture Assessment: Radiography vs. Posture and Their Clinical Comparison,2024,"Background:Forward head posture (FHP) and altered cervical lordotic curvatures are common spine displacements often associated with neck pain and disability. Two primary categories for determining FHP exist: radiographic and postural measurements.Methods:This study investigated the correlation between the craniovertebral angle (CVA), the radiographically measured C2–C7 sagittal vertical axis (SVA), and cervical lordosis (absolute rotation angle: ARA C2–C7) in a sample of participants with chronic myofascial pain (CMP). In 120 participants, we performed both a postural measurement of the CVA and a lateral cervical radiograph, where the C2–C7 SVA and ARA C2–C7 were measured. A linear-regression Rvalue to assess the correlation between the CVA, C2–C7 SVA, and ARA C2–C7 was sought.Results:A statistically significant weak linear fit was identified (Spearman’s r = 0.549; R= 0.30,p< 0.001) between the CVA and C2–C7 SVA, having considerable variation between the two measures. A statistically significant linear fit (very weak) was identified for the lordosis ARA C2–C7 and the CVA: Spearman’s r = 0.524; R= 0.275;p< 0.001. A value of 50° for the CVA corresponded to a value of 20 mm for the C2–C7 SVA on an X-ray.Conclusion:While the CVA and radiographic C2–C7 SVA are weakly correlated in an individual, they seem to represent different aspects of sagittal cervical balance. The CVA cannot replace radiographically measured cervical lordosis. We recommend that more emphasis be given to radiographic measures of sagittal cervical alignment than the CVA when considering patient interventions."
1,Effect of pain on cranio-cervico-mandibular function and postural stability in people with temporomandibular joint disorders,2024,"Background: Neck and jaw pain is common and is associated with jaw functional limitations, postural stability, muscular endurance, and proprioception. This study aimed to investigate the effect of jaw and neck pain on craniocervico- mandibular functions and postural stability in patients with temporomandibular joint disorders (TMJDs).\nMethods: Fifty-two patients with TMJDs were included and assessed using Fonseca's Questionnaire and the Helkimo Clinical Dysfunction Index. An isometric strength test was performed for the TMJ depressor and cervical muscles. The TMJ position sense (TMJPS) test and cervical joint position error test (CJPET) were employed for proprioception. Total sway degree was obtained for the assessment of postural stability. Deep neck flexor endurance (DNFE) was assessed using the craniocervical flexion test. The mandibular function impairment questionnaire (MFIQ) was employed to assess mandibular function, and the craniovertebral angle (CVA) was measured for forward head posture.\nResults: Jaw and neck pain negatively affected CVA (R2 = 0.130), TMJPS (R2 = 0.286), DNFE (R2 = 0.355), TMJ depressor (R2 = 0.145), cervical flexor (R2 = 0.144), and extensor (R2 = 0.148) muscle strength. Jaw and neck pain also positively affected CJPET for flexion (R2 = 0.116) and extension (R2 = 0.146), as well as total sway degree (R2 = 0.128) and MFIQ (R2 = 0.230).\nConclusions: Patients with painful TMJDs, could have impaired muscle strength and proprioception of the TMJ and cervical region. The jaw and neck pain could also affect postural stability, and the endurance of deep neck flexors as well as mandibular functions in TMJDs."
2,Efficacy of posture correction band vs. McKenzie's exercises on pulmonary function and chest expansion in asymptomatic population with forward head posture,2024,"Background Poor posture and sedentary lifestyle cause Forward Head Posture (FHP). To correct this, a Correction Band (PCB) is commonly used. However, the efficacy of PCB vs. McKenzie's Exercises on pulmonary function chest expansion in asymptomatic individuals with FHP was not known. Objective: Th

## 데이터 전처리

### 결측치 처리 
- abstract가 없는 값은 제목으로 대체
- abstract과 제목이 없는 경우 삭제

In [380]:
FHP_article.query("abstract_list==''").count()

title            312
artical_date     312
abstract_list    312
dtype: int64

In [381]:
FHP_article.loc[FHP_article.query("abstract_list==''").index,'abstract_list'] = FHP_article.loc[FHP_article.query("abstract_list==''").index,'title']

In [382]:
FHP_article.query("abstract_list==''")

,title,artical_date,abstract_list
394,,2023,
479,,2022,
726,,2022,
762,,2021,
1207,,2018,
1302,,2017,
1588,,2013,
1640,,2010,
1646,,2010,


In [383]:
FHP_article = FHP_article.query("abstract_list!=''")

### 중복값 제거

In [384]:
FHP_article = FHP_article.drop_duplicates(subset='title', keep='first', inplace=False, ignore_index=False)

### 발행일 변환

In [385]:
# def date_year(raw):
#     date_format = raw.split(" ")[0]
#     return date_format
# def date_month(raw):
#     if len(raw.split(" ")) >= 2:
#         date_format = raw.split(" ")[1]
#     else:
#         date_format = None
#     return date_format
# def date_date(raw):
#     if len(raw.split(" ")) >= 3:
#         date_format = raw.split(" ")[2]
#     else:
#         date_format = None
#     return date_format
# def word_split(raw,word_list):
#     word_list.append(raw.split())
#     return word_list

In [386]:
# FHP_article.loc[:,'date_year']=FHP_article['artical_date'].apply(date_year)
# FHP_article.loc[:,'date_month']=FHP_article['artical_date'].apply(date_month)
# FHP_article.loc[:,'date_date']=FHP_article['artical_date'].apply(date_date)

### 정규화 : 대소문자 통합

In [387]:
def word_lower(raw):
    abstract_list = raw.split()
    modified_list = []
    for abstract_word in abstract_list :
         modified_list.append(abstract_word.lower())
    return " ".join(modified_list)

In [388]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_list'].apply(word_lower)

In [389]:
FHP_article.loc[2,'abstract_clean']

"background poor posture and sedentary lifestyle cause forward head posture (fhp). to correct this, a correction band (pcb) is commonly used. however, the efficacy of pcb vs. mckenzie's exercises on pulmonary function chest expansion in asymptomatic individuals with fhp was not known. objective: this study aimed to determine pulmonary population fhp. methodology: a randomized control trial conducted forty-two subjects subjects were divided two groups. g1 group educated as per exercises perform once daily for month. the test this performed before after mckenzie exercises. g2 wore 2 h month their pft recorded trial. fvc, fev1, fev1/fvc ratio, pefr chest measured. results: p-value ratio between groups (treatment group) significant 0.000, 0.000 0.02 respectively. non-significant 0.553, 0.493 0.699 at axillary, 4th intercostal xiphisternum level (control 0.682, 0.149, 0.424 0.414 also 0.853, 0.651 0.763 level. conclusion: concluded that there effects both mc’kenzie greater difference seen t

### 특수문자, 숫자, '/n', '소제목' 제거

In [390]:
import re
def word_cleaning(raw):
    raw = raw.replace("\n"," ")
    raw = raw.replace("background:"," ")
    raw = raw.replace("purpose:"," ")
    raw = raw.replace("objectives:"," ")
    raw = raw.replace("objective:"," ")
    raw = raw.replace("design:"," ")
    raw = raw.replace("methods:"," ")
    raw = raw.replace("method:"," ")
    raw = raw.replace("results:"," ")
    raw = raw.replace("result:"," ")
    raw = raw.replace("discussion:"," ")
    raw = raw.replace("conclusions:"," ")
    raw = raw.replace("interpretation:"," ")
    raw = raw.replace("conclusion:"," ")
    raw = raw.replace("significance:"," ")
    raw = raw.replace("keywords:"," ")
    raw = raw.replace("fhp"," ")
    raw = raw.replace("forward head posture"," ")
    raw = raw.replace("head forward posture"," ")
    patterns = '([^a-z|A-Z|\'| ])'
    text_regex = re.sub(pattern = patterns, repl = "",string=raw)
    return text_regex

In [391]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_clean'].apply(word_cleaning)

In [392]:
FHP_article.loc[1789,'abstract_clean']

'                stabilizer               flexfsr                        cva cra                          stabilizer               cva cra         cva cra            cva   iqr       cra   iqr    cva           cra                  stabilizer             the purpose of this study was designed to investigate the effect of newly developed cervicalretraction assistive tool and a existing exercise assistive tool stabilizer on the neck alignment of an adult with a   for six weeks to develop the cervicalretraction assistive tool the flexfsr sensors are mounted on the pillow and the information is received through the bluetooth module connected to the arduino after the conditional statement operation is performed within the application the vibration is output when the event processing is completed the subjects of this study were selected from  subjects who showed   through cranio vertebral anglecva and cranial rotation anglecra subjects were randomly assigned to an experimental group n and a co

### 단어 토큰화

In [393]:
def word_tokenizing(raw):
    tokenizer = RegexpTokenizer("[\s]+", gaps=True) 
    tokens = tokenizer.tokenize(raw)
    tags = nltk.pos_tag(tokens)
    # nouns_list = [t for t in tags if t[1] in  ['NN','NNP','NNS']]
    nouns_list = [t for t in tags if t[1] in  ['JJ','JJR','NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    # nouns_list = [t for t in tags if t[1] in  ['NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    return nouns_list
# def word_tokenizing(raw):
#     tokenizer = RegexpTokenizer("[\s]+", gaps=True) 
#     text_list = []
#     for raw_senetence in raw:
#         tokens = tokenizer.tokenize(raw_senetence)
#         tags = nltk.pos_tag(tokens)
#         text_list.append(nouns_list)
#     return text_list

<>:2: DeprecationWarning: invalid escape sequence '\s'
<>:2: DeprecationWarning: invalid escape sequence '\s'
/tmp/ipykernel_9924/1482873834.py:2: DeprecationWarning: invalid escape sequence '\s'
  tokenizer = RegexpTokenizer("[\s]+", gaps=True)


In [394]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_clean'].apply(word_tokenizing)

In [395]:
FHP_article['abstract_token']

0                                                                                           [(altered, VBD), (cervical, JJ), (lordotic, JJ), (curvatures, NNS), (are, VBP), (common, JJ), (spine, JJ), (displacements, NNS), (associated, VBN), (neck, NN), (pain, NN), (disability, NN), (primary, JJ), (categories, NNS), (determining, VBG), (exist, JJ), (radiographic, JJ), (postural, JJ), (measurements, NNS), (study, NN), (investigated, VBD), (correlation, NN), (craniovertebral, JJ), (angle, NN), (cva, VBD), (measured, VBN), (cc, NN), (sagittal, JJ), (vertical, JJ), (axis, NN), (sva, NN), (cervical, JJ), (lordosis, NN), (absolute, NN), (rotation, NN), (angle, NN), (ara, NN), (cc, NN), (sample, NN), (participants, NNS), (chronic, JJ), (myofascial, JJ), (pain, NN), (cmp, NN), (participants, NNS), (performed, VBD), (postural, JJ), (measurement, NN), (cva, NN), (lateral, JJ), (cervical, JJ), (radiograph, NN), (cc, NN), (sva, NN), (ara, NN), (cc, NN), (were, VBD), (measured, VBN), (linearregressio

### 표제어 추출

In [396]:
def Lemmatization(raw):
    lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    for word in raw:
        if word[1][0].lower() in ['v','n']:
            lemmatized_word = lemmatizer.lemmatize(word[0],word[1][0].lower())
            if lemmatized_word != 'be':
                lemmatized_list.append(lemmatized_word)
        else:
            lemmatized_list.append(word[0])
    return lemmatized_list
# def Lemmatization(raw):
#     lemmatizer = WordNetLemmatizer()
#     text_list = []
#     for raw_senetence in raw:
#         lemmatized_list = []
#         for word in raw_senetence:
#             if word[1][0].lower() in ['v','n']:
#                 lemmatized_word = lemmatizer.lemmatize(word[0],word[1][0].lower())
#                 lemmatized_list.append(lemmatized_word)
#         text_list.append(lemmatized_list)
#     return text_list

In [397]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_token'].apply(Lemmatization)

In [398]:
FHP_article.loc[:,'abstract_token']

0                                                                                        [alter, cervical, lordotic, curvature, common, spine, displacement, associate, neck, pain, disability, primary, category, determine, exist, radiographic, postural, measurement, study, investigate, correlation, craniovertebral, angle, cva, measure, cc, sagittal, vertical, axis, sva, cervical, lordosis, absolute, rotation, angle, ara, cc, sample, participant, chronic, myofascial, pain, cmp, participant, perform, postural, measurement, cva, lateral, cervical, radiograph, cc, sva, ara, cc, measure, linearregression, rvalue, assess, correlation, cva, cc, sva, ara, cc, seek, significant, weak, linear, fit, identify, spearmans, r, r, p, cva, cc, sva, have, considerable, variation, measure, significant, linear, fit, weak, identify, lordosis, ara, cc, cva, spearmans, r, r, p, value, cva, correspond, value, mm, ...]
1                 [neck, jaw, pain, common, associate, jaw, functional, limitation, postural,

In [399]:
def join_words(raw):
    sentence = ' '.join(raw)
    return sentence
# def join_words(raw):
#     text_list = []
#     for raw_senetence in raw:
#         sentence = ' '.join(raw_senetence)
#         text_list.append(sentence)
#     return text_list

In [400]:
FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_token'].apply(join_words)

In [401]:
FHP_article

,title,artical_date,abstract_list,abstract_clean,abstract_token,abstract_join
0,Two Methods of Forward Head Posture Assessment: Radiography vs. Posture and Their Clinical Comparison,2024,"Background:Forward head posture (FHP) and altered cervical lordotic curvatures are common spine displacements often associated with neck pain and disability. Two primary categories for determining FHP exist: radiographic and postural measurements.Methods:This study investigated the correlation between the craniovertebral angle (CVA), the radiographically measured C2–C7 sagittal vertical axis (SVA), and cervical lordosis (absolute rotation angle: ARA C2–C7) in a sample of participants with chronic myofascial pain (CMP). In 120 participants, we performed both a postural measurement of the CVA and a lateral cervical radiograph, where the C2–C7 SVA and ARA C2–C7 were measured. A linear-regression Rvalue to assess the correlation between the CVA, C2–C7 SVA, and ARA C2–C7 was sought.Results:A statistically significant weak linear fit was identified (Spearman’s r = 0.549; R= 0.30,p< 0.001) between the CVA and C2–C7 SVA, having considerable variation between the two measures. A statistically significant linear fit (very weak) was identified for the lordosis ARA C2–C7 and the CVA: Spearman’s r = 0.524; R= 0.275;p< 0.001. A value of 50° for the CVA corresponded to a value of 20 mm for the C2–C7 SVA on an X-ray.Conclusion:While the CVA and radiographic C2–C7 SVA are weakly correlated in an individual, they seem to represent different aspects of sagittal cervical balance. The CVA cannot replace radiographically measured cervical lordosis. We recommend that more emphasis be given to radiographic measures of sagittal cervical alignment than the CVA when considering patient interventions.",and altered cervical lordotic curvatures are common spine displacements often associated with neck pain and disability two primary categories for determining exist radiographic and postural measurements this study investigated the correlation between the craniovertebral angle cva the radiographically measured cc sagittal vertical axis sva and cervical lordosis absolute rotation angle ara cc in a sample of participants with chronic myofascial pain cmp in participants we performed both a postural measurement of the cva and a lateral cervical radiograph where the cc sva and ara cc were measured a linearregression rvalue to assess the correlation between the cva cc sva and ara cc was sought a statistically significant weak linear fit was identified spearmans r r p between the cva and cc sva having considerable variation between the two measures a statistically significant linear fit very weak was identified for the lordosis ara cc and the cva spearmans r r p a value of for the cva corresponded to a value of mm for the cc sva on an xray while the cva and radiographic cc sva are weakly correlated in an individual they seem to represent different aspects of sagittal cervical balance the cva cannot replace radiographically measured cervical lordosis we recommend that more emphasis be given to radiographic measures of sagittal cervical alignment than the cva when considering patient interventions,"[alter, cervical, lordotic, curvature, common, spine, displacement, associate, neck, pain, disability, primary, category, determine, exist, radiographic, postural, measurement, study, investigate, correlation, craniovertebral, angle, cva, measure, cc, sagittal, vertical, axis, sva, cervical, lordosis, absolute, rotation, angle, ara, cc, sample, participant, chronic, myofascial, pain, cmp, participant, perform, postural, measurement, cva, lateral, cervical, radiograph, cc, sva, ara, cc, measure, linearregression, rvalue, assess, correlation, cva, cc, sva, ara, cc, seek, significant, weak, linear, fit, identify, spearmans, r, r, p, cva, cc, sva, have, considerable, variation, measure, significant, linear, fit, weak, identify, lordosis, ara, cc, cva, spearmans, r, r, p, value, cva, correspo

### 유사한 단어 묶기

In [402]:
def word_cleaning(raw):
    raw = raw.replace("chronic neck pain","cnp")
    raw = raw.replace("craniovertebral angle","cva")
    
    return raw

In [403]:
FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_join'].apply(word_cleaning)

In [404]:
len(FHP_article)

1098

In [405]:
FHP_article.isnull().sum()

title             0
artical_date      0
abstract_list     0
abstract_clean    0
abstract_token    0
abstract_join     0
dtype: int64

## 학습데이터, 테스트 데이터 분류

In [406]:
from sklearn.model_selection import train_test_split
FHP_article_train,FHP_article_test = train_test_split(FHP_article,test_size=0.4,random_state=42)

## 키워드 분류하기

### TfidfVectorizer 최적화

In [447]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(max_df=0.60,min_df=0.2,ngram_range=(1,2), stop_words = ['use','have','show'])
features_train = tfidfVectorizer.fit_transform(FHP_article_train['abstract_join'])
dictionary_list = tfidfVectorizer.get_feature_names_out()
dictionary_list

array(['angle', 'cervical', 'change', 'compare', 'control', 'cva',
       'difference', 'effect', 'exercise', 'group', 'head', 'improve',
       'increase', 'intervention', 'investigate', 'measure', 'method',
       'muscle', 'neck', 'pain', 'patient', 'perform', 'postural',
       'posture', 'purpose', 'result', 'shoulder', 'significant',
       'significant difference', 'subject', 'test', 'upper', 'week'],
      dtype=object)

### gridsearch

In [489]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

# LDA 모델을 위한 파라미터 그리드 생성
param_grid = {
    'n_components': [3],
    'learning_decay': [0.7],
    'doc_topic_prior': [1.0],  # 문서-주제 사전확률 (알파)
    'topic_word_prior': [1.0]  # 단어-주제 사전확률 (베타)
}

# LatentDirichletAllocation 모델 인스턴스 생성
lda = LatentDirichletAllocation()

# GridSearchCV 인스턴스 생성
model = GridSearchCV(lda, param_grid, n_jobs=-1, cv=5)

# 데이터는 여기에 당신의 데이터셋을 사용하세요
# X는 문서-단어 행렬이어야 합니다. 예를 들어, CountVectorizer나 TfidfVectorizer를 사용하여 생성할 수 있습니다.
model.fit(features_train)

# 최적 모델
grid_best_model = model.best_estimator_

# 최적의 파라미터와 점수를 출력
print("Best Model's Params: ", model.best_params_)
print("Best Log Likelihood Score: ", model.best_score_)
print("perplexity",grid_best_model.perplexity(features_train) )


Best Model's Params:  {'doc_topic_prior': 1.0, 'learning_decay': 0.7, 'n_components': 3, 'topic_word_prior': 1.0}
Best Log Likelihood Score:  -1314.1303723693895
perplexity 42.13631297539492


In [490]:
grid_best_model.fit(features_train)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_grid = [['Topic1', 'Topic2','Topic3','Topic4','Topic5'],topics_list]
df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
df_topics_keywords_grid=df_topics_keywords_grid.T
df_topics_keywords_grid

,0,1
0,Topic1,posture cervical neck pain shoulder head angle patient subject result
1,Topic2,muscle effect postural posture pain group neck cervical subject change
2,Topic3,exercise group cva week significant difference intervention control muscle cervical
3,Topic4,None
4,Topic5,None


In [491]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features_train,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

AssertionError: Term frequencies and vocabulary are of different sizes, 33 != 30.

In [ ]:
topics_output = grid_best_model.transform(features_train)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

### test set에 적용해보기

In [ ]:
features_test = tfidfVectorizer.fit_transform(FHP_article_test['abstract_join'])
grid_best_model.fit(features_test)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
#     print(df_topics[1].values[:10])
#     topics_text = ' '.join(df_topics[1].values[:9])# 시리즈 형식으로 출력 get values from series / index 
#     topics_list.append(topics_text)
# topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
# df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
# df_topics_keywords_grid=df_topics_keywords_grid.T
# df_topics_keywords_grid

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features_test,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [ ]:
topics_output = grid_best_model.transform(features_test)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

### 전체 데이터에 적용

In [500]:
features = tfidfVectorizer.fit_transform(FHP_article['abstract_join'])
grid_best_model.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
#     # topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index 
#     # topics_list.append(topics_text)
# topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
# df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
# df_topics_keywords_grid=df_topics_keywords_grid.T
# df_topics_keywords_grid

In [501]:
df_datas_0 = [grid_best_model.components_[0], dictionary_list]
df_topics_0 = pd.DataFrame(data=df_datas_0).T
df_topics_0 = df_topics_0.dropna()
df_topics_0 = df_topics_0.sort_values(0, ascending=False).reset_index()
df_topics_0.rename(columns = {1 : 'first_components'}, inplace = True)
df_topics_0.rename(columns = {0 : 'score'}, inplace = True)
df_datas_1 = [grid_best_model.components_[1], dictionary_list]
df_topics_1 = pd.DataFrame(data=df_datas_1).T
df_topics_1 = df_topics_1.dropna()
df_topics_1 = df_topics_1.sort_values(0, ascending=False).reset_index()
df_topics_1.rename(columns = {1 : 'second_components'}, inplace = True)
df_topics_1.rename(columns = {0 : 'score'}, inplace = True)
df_datas_2 = [grid_best_model.components_[2], dictionary_list]
df_topics_2 = pd.DataFrame(data=df_datas_2).T
df_topics_2 = df_topics_2.dropna()
df_topics_2 = df_topics_2.sort_values(0, ascending=False).reset_index()
df_topics_2.rename(columns = {1 : 'third_components'}, inplace = True)
df_topics_2.rename(columns = {0 : 'score'}, inplace = True)
df_topic = pd.concat([df_topics_0.loc[:20,['score','first_components']],df_topics_1.loc[:20,['score','second_components']],df_topics_2.loc[:20,['score','third_components']]],axis=1)
df_topic

,score,first_components,score,second_components,score,third_components
0,146.658091,group,113.517922,cervical,89.096693,posture
1,80.32563,muscle,109.199413,pain,66.605734,angle
2,68.791226,exercise,82.498081,neck,66.217393,head
3,53.78949,control,65.404307,patient,60.446356,neck
4,52.393278,subject,44.233576,cva,56.695425,significant difference
5,48.588167,difference,40.809828,effect,56.291419,exercise
6,45.423472,shoulder,39.714225,muscle,47.764209,method
7,36.61769,cva,28.638412,exercise,39.779377,result
8,36.3301,posture,28.09977,measure,37.702768,muscle
9,35.149399,postural,27.001168,posture,37.030502,measure


In [502]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [503]:
topics_output = grid_best_model.transform(features)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
0    458
1    324
2    316
Name: count, dtype: int64

In [504]:
df_topics_score['dominant_topic_number']

0       1
1       1
2       0
3       0
4       0
       ..
1093    0
1094    2
1095    1
1096    2
1097    0
Name: dominant_topic_number, Length: 1098, dtype: int64

## 토픽 분석하기

### topic별로 분류하기

In [505]:
FHP_article = FHP_article.reset_index()
FHP_article.loc[:,'article_topic'] = df_topics_score['dominant_topic_number']

In [506]:
FHP_article.isnull().sum()

level_0           0
index             0
title             0
artical_date      0
abstract_list     0
abstract_clean    0
abstract_token    0
abstract_join     0
article_topic     0
dtype: int64

### topic 별 많이 등장하는 단어 추출

In [507]:
FHP_article.query("article_topic == 0")

,level_0,index,title,artical_date,abstract_list,abstract_clean,abstract_token,abstract_join,article_topic
2,2,2,Efficacy of posture correction band vs. McKenzie's exercises on pulmonary function and chest expansion in asymptomatic population with forward head posture,2024,"Background Poor posture and sedentary lifestyle cause Forward Head Posture (FHP). To correct this, a Correction Band (PCB) is commonly used. However, the efficacy of PCB vs. McKenzie's Exercises on pulmonary function chest expansion in asymptomatic individuals with FHP was not known. Objective: This study aimed to determine Pulmonary population FHP. Methodology: A Randomized control trial conducted forty-two subjects Subjects were divided two groups. G1 group educated as per exercises perform once daily for month. The test this performed before after McKenzie exercises. G2 wore 2 h month their PFT recorded trial. FVC, FEV1, FEV1/FVC ratio, PEFR Chest measured. Results: P-value ratio between groups (treatment group) significant 0.000, 0.000 0.02 respectively. non-significant 0.553, 0.493 0.699 at axillary, 4th intercostal xiphisternum level (control 0.682, 0.149, 0.424 0.414 also 0.853, 0.651 0.763 level. Conclusion: concluded that there effects both Mc’Kenzie greater difference seen terms P-values treatment group.",background poor posture and sedentary lifestyle cause to correct this a correction band pcb is commonly used however the efficacy of pcb vs mckenzie's exercises on pulmonary function chest expansion in asymptomatic individuals with was not known this study aimed to determine pulmonary population methodology a randomized control trial conducted fortytwo subjects subjects were divided two groups g group educated as per exercises perform once daily for month the test this performed before after mckenzie exercises g wore h month their pft recorded trial fvc fev fevfvc ratio pefr chest measured pvalue ratio between groups treatment group significant respectively nonsignificant at axillary th intercostal xiphisternum level control also level concluded that there effects both mckenzie greater difference seen terms pvalues treatment group,"[background, poor, posture, sedentary, lifestyle, cause, correct, correction, band, pcb, use, efficacy, pcb, v, mckenzie's, exercise, pulmonary, function, expansion, asymptomatic, individual, know, study, aim, determine, pulmonary, population, methodology, randomize, control, trial, conduct, fortytwo, subject, subject, divide, group, g, group, educate, exercise, perform, month, test, perform, mckenzie, exercise, g, h, month, pft, record, trial, fvc, fev, fevfvc, ratio, pefr, chest, measure, pvalue, ratio, group, treatment, group, significant, nonsignificant, axillary, th, intercostal, xiphisternum, level, control, level, conclude, effect, mckenzie, greater, difference, see, term, pvalues, treatment, group]",background poor posture sedentary lifestyle cause correct correction band pcb use efficacy pcb v mckenzie's exercise pulmonary function expansion asymptomatic individual know study aim determine pulmonary population methodology randomize control trial conduct fortytwo subject subject divide group g group educate exercise perform month test perform mckenzie exercise g h month pft record trial fvc fev fevfvc ratio pefr chest measure pvalue ratio group treatment group significant nonsignificant axillary th intercostal xiphisternum level control level conclude effect mckenzie greater difference see term pvalues treatment group,0
3,3,3,Effects of Cervical Mobilization on Balance and Gait Parameters in Individuals With Stroke: A Randomized Controlled Trial,2024,"Stroke is a significant health problem that may result in long-term functional deficits. Balance and walking problems are among the most common post-stroke deficits, and they may negatively affect quality of life. Our aim in this study was to investigate the effects of cervical mobilization on balance and gait parameters after stroke. Participants were 24 adults (a

In [508]:
len(FHP_article.query("article_topic == 1"))

324

In [509]:
len(FHP_article.query("article_topic == 2"))

316

In [510]:
FHP_article.to_csv('FHP_article.csv',index=False,encoding='utf-8')

## 결론
- 논문
- 중복 논문을 삭제하고, 결측치를 제거하니 데이터가 1000개정도밖에 남지 않음

### Count vectorizer

In [430]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words=[], ngram_range=(1, 2))

In [431]:
FHP_article.query("article_topic == 2")['abstract_join']

1                                                                                                                                                                                                                                                                         neck jaw pain common associate jaw functional limitation postural stability muscular endurance proprioception study aim investigate effect jaw neck pain craniocervico mandibular function postural stability patient temporomandibular joint disorder tmjds fiftytwo patient tmjds include assess use fonseca's questionnaire helkimo clinical dysfunction index isometric strength test perform tmj depressor cervical muscle tmj position sense tmjps test cervical joint position error test cjpet employ proprioception total degree obtain assessment postural stability deep neck flexor endurance dnfe assess use craniocervical flexion mandibular function impairment questionnaire mfiq employ assess mandibular function cva cva measure jaw neck 

In [432]:
count_vect.fit(FHP_article['abstract_join'])

CountVectorizer(ngram_range=(1, 2), stop_words=[])

In [433]:
count_vect.vocabulary_

{'alter': 2146,
 'cervical': 7366,
 'lordotic': 32074,
 'curvature': 12693,
 'common': 9357,
 'spine': 54047,
 'displacement': 16097,
 'associate': 4241,
 'neck': 36986,
 'pain': 39419,
 'disability': 15695,
 'primary': 44716,
 'category': 6884,
 'determine': 14544,
 'exist': 19851,
 'radiographic': 46327,
 'postural': 43285,
 'measurement': 33645,
 'study': 55584,
 'investigate': 29416,
 'correlation': 11787,
 'cva': 12790,
 'measure': 33305,
 'cc': 7042,
 'sagittal': 49965,
 'vertical': 63385,
 'axis': 4785,
 'sva': 57038,
 'lordosis': 32000,
 'absolute': 219,
 'rotation': 49672,
 'angle': 2695,
 'ara': 3666,
 'sample': 50066,
 'participant': 40196,
 'chronic': 8334,
 'myofascial': 36707,
 'cmp': 8890,
 'perform': 41364,
 'lateral': 30848,
 'radiograph': 46304,
 'linearregression': 31713,
 'rvalue': 49892,
 'assess': 3919,
 'seek': 51147,
 'significant': 52733,
 'weak': 63895,
 'linear': 31701,
 'fit': 21432,
 'identify': 26337,
 'spearmans': 53883,
 'have': 24633,
 'considerable': 1

In [434]:
dtm_topic_0 = count_vect.transform(FHP_article.query("article_topic == 0")['abstract_join'])
dtm_array_0 = dtm_topic_0.toarray()
df_dtm_0 = pd.DataFrame(dtm_array_0, columns = vocabs)
df_first_topic = df_dtm_0.sum().sort_values(ascending=False)[:20]
df_first_topic = df_first_topic.reset_index()
df_first_topic.rename(columns = {'index' : 'first'}, inplace = True)
df_first_topic.rename(columns = {0 : 'count'}, inplace = True)

In [435]:
dtm_topic_1 = count_vect.transform(FHP_article.query("article_topic == 1")['abstract_join'])
dtm_array_1 = dtm_topic_1.toarray()
df_dtm_1 = pd.DataFrame(dtm_array_1, columns = vocabs)
df_second_topic = df_dtm_1.sum().sort_values(ascending=False)[:20]
df_second_topic = df_second_topic.reset_index()
df_second_topic.rename(columns = {'index' : 'second'}, inplace = True)
df_second_topic.rename(columns = {0 : 'count'}, inplace = True)

In [436]:
dtm_topic_2 = count_vect.transform(FHP_article.query("article_topic == 2")['abstract_join'])
dtm_array_2 = dtm_topic_2.toarray()
df_dtm_2 = pd.DataFrame(dtm_array_2, columns = vocabs)
df_third_topic = df_dtm_2.sum().sort_values(ascending=False)[:20]
df_third_topic = df_third_topic.reset_index()
df_third_topic.rename(columns = {'index' : 'third'}, inplace = True)
df_third_topic.rename(columns = {0 : 'count'}, inplace = True)

In [437]:
vocabs = count_vect.get_feature_names_out()
vocabs[:2]
vocabs.shape

(65198,)

In [438]:
df_topic = pd.concat([df_first_topic,df_second_topic,df_third_topic],axis=1)

In [439]:
df_topic

,first,count,second,count,third,count
0,group,2128,posture,657,muscle,821
1,exercise,1220,study,466,neck,747
2,significant,830,head,424,cervical,615
3,study,803,use,412,pain,591
4,cva,692,angle,397,study,480
5,muscle,649,neck,256,patient,442
6,difference,629,have,239,use,382
7,neck,622,muscle,236,exercise,292
8,week,530,shoulder,222,group,287
9,control,529,cervical,219,posture,255
